# Tarea 2: Dockerizar Apps de Dash

Dockerize los dashboard de los dos ejemplos estudiados en la sección Dos ejemplos de Dash. Esto es. Crear dos imágenes Docker para los Dash asociados con: Dash App para Mapas y Dash App Financiera. Realice cambios en el (back/front end) de los dos Dashboards. Por ejemplo, agréguele a los dashboard nuevos estilos y operaciones que puedan mejorar la visualización. Tenga en cuenta los diferentes templates usados en esta sección.

## Dash App para mapas

Para la primera app se utilizá un conjunto de datos que contiene información sobre las enfermedades que afectan a ciertas colonias de abejas, en determinados estados en USA. la aplicación a desplegar mostrará como se distribuye en los distintos estados de Estados Unidos, el porcentaje promedio de colonias de abejas afectadas por enfermedad.

A continuación se muestra el codigo completo que se usó para crear la aplicación:

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output

app = Dash(__name__)

df = pd.read_csv("https://raw.githubusercontent.com/lihkir/Uninorte/main/AppliedStatisticMS/DataVisualizationRPython/Lectures/Python/PythonDataSets/intro_bees.csv")

df = df.groupby(['State', 'ANSI', 'Affected by', 'Year', 'state_code'])[['Pct of Colonies Impacted']].mean()
df.reset_index(inplace=True)
print(df[:5])

app.layout = html.Div([
    html.H1("Porcentaje promedio de colonias de abejas afectadas por enfermedad", style={'text-align': 'center'}),
    html.Div(
    dcc.Slider(id="slct_year",
                 value=2015,
                 marks={2015: '2015',
                        2016: '2016',
                        2017: '2017',
                        2018: '2018'}
                 ), hidden=True),
    html.Div(id='output_container', children=[]),
    html.Br(),
    dcc.Graph(id='my_bee_map', figure={})
])

@app.callback(
    # [Output(component_id='output_container', component_property='children'),
    Output(component_id='my_bee_map', component_property='figure',),
    [Input(component_id='slct_year', component_property='value')]
)
def display_animated_graph(soption_slctd):
    
    dff = df.copy()
    # dff = dff[dff["Year"] == option_slctd]
    dff = dff[dff["Affected by"] == "Varroa_mites"]
    
    animation = px.choropleth(
            data_frame=dff,
            locationmode='USA-states',
            locations='state_code',
            scope="usa",
            color='Pct of Colonies Impacted',
            hover_data=['State', 'Pct of Colonies Impacted'],
            color_continuous_scale=px.colors.sequential.deep,
            labels={'Pct of Colonies Impacted': '% of Bee Colonies'},
            template='plotly_white',
            animation_frame='Year',
            animation_group="State"
            
        )
    
    
    return animation

if __name__ == '__main__':
    # app.run_server(debug=True) 
    app.run_server(debug=True, host='0.0.0.0', port=9000)

La aplicación permite ver a través de una animación, como se distribuye el porcentaje de colonias afectadas a lo largo de los años. Con el gráfico se puede reproducir la animación o escoger manualmente el año que se desea visualizar. 

La aplicación al ser ejecutada, luce de la siguiente manera:


```{figure} C:/Users/kaes1/Documents/DataViz/maps.png
---
height: 400px
name: directive-fig
---
App Mapas
```

El paso a paso para realizar la dockerización de la app es el siguiente:



1. Crear un entorno virtual de Python:

In [ ]:
conda create --name maps_venv python=3.9

2. Ejecutar la aplicación Dash dentro del entorno virtual:

In [ ]:
python index.py

Para este paso se descomenta la linea de codigo "# app.run_server(debug=True)", y se comenta " app.run_server(debug=True, host='0.0.0.0', port=9000)". En este paso se verifican las librerias que se deben instalar en el ambiente para que la app ejecute correctamente. Tambien se debe crear una carpeta para ubicar solo el archivo .py, y los demás archivos necesarios para crear la imagen de la app.

3. Crear un archivo de requerimientos:

In [ ]:
pip list --format=freeze > requirements.txt

4. Crear un archivo Dockerfile en la carpeta donde se encuentra el archivo .py:

In [ ]:
FROM python:3.9.17

COPY ./requirements.txt /requirements.txt

RUN pip install --upgrade pip
RUN pip install --no-cache-dir --upgrade -r /requirements.txt

COPY ./index.py /index.py

CMD ["python", "index.py"]

5. Crear un archivo README:

In [ ]:
# Execute the following commands to Dockerize the Dash application

docker build -t dash_maps .
docker run -h localhost -p 9002:9000 -d --name dash_maps dash_maps

6. Crear la imagen Docker:

In [ ]:
docker build -t dash_maps .

7. Ejecutar la imagen Docker:

In [ ]:
docker run -h localhost -p 9002:9000 -d --name dash_maps dash_maps

## Dash App Financiera

Para esta app se mostrarán datos del precio de distintos stocks, utilizanfo el indicador de análisis técnico, Bollinger Bands. Las Bandas de Bollinger son un indicador técnico que muestra la volatilidad de los precios de las acciones. Se pueden utilizar para identificar posibles oportunidades de compra y venta, ya que cuando los precios se encuentran fuera de las bandas, es probable que haya un cambio en la tendencia.

A continuación se muestra el codigo completo que se usó para crear la aplicación:

In [ ]:
from dash import Dash, dcc, html, Input, Output, State
import colorlover as cl
import pandas as pd

app = Dash(__name__)

app.scripts.config.serve_locally = False

colorscale = cl.scales['9']['qual']['Paired']

df = pd.read_csv('https://raw.githubusercontent.com/lihkir/Uninorte/main/AppliedStatisticMS/DataVisualizationRPython/Lectures/Python/PythonDataSets/dash-stock-ticker-demo.csv')

app.layout = html.Div([
    html.Div([
        html.H2('Finance Explorer',
                style={'display': 'inline',
                       'float': 'left',
                       'font-size': '2.65em',
                       'margin-left': '7px',
                       'font-weight': 'bolder',
                       'font-family': 'system-ui',
                       'color': "black",
                       'margin-top': '20px',
                       'margin-bottom': '0'
                       }),
        html.Img(src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/ff/Logo_uninorte_colombia.jpg/1200px-Logo_uninorte_colombia.jpg",
                style={
                    'height': '100px',
                    'float': 'right'
                },
        ),
    ]),
    dcc.Dropdown(
        id='stock-ticker-input',
        options=[{'label': html.Span(s[0], style={'color': 'black', 'font-size': 15}), 'value': str(s[1])}
                 for s in zip(df.Stock.unique(), df.Stock.unique())],
        value=['YHOO', 'GOOGL'],
        multi=True
    ),
    dcc.Checklist(
        id='rangeslider-toggle',
        options=[{'label': html.Span('Mostrar Rangeslider', style={'color': 'black', 'font-size': 15, 'font-family':'system-ui'}), 'value': 'rangeslider'}],
        value=[],  
    ),
    html.Div(id='graphs')
], className="container")

def bbands(price, window_size=10, num_of_std=5):
    rolling_mean = price.rolling(window=window_size).mean()
    rolling_std = price.rolling(window=window_size).std()
    upper_band = rolling_mean + (rolling_std * num_of_std)
    lower_band = rolling_mean - (rolling_std * num_of_std)
    return rolling_mean, upper_band, lower_band

@app.callback(Output('graphs', 'children'),
              [Input('stock-ticker-input', 'value')],
              [Input('rangeslider-toggle', 'value')])  
def update_graph(tickers, rangeslider_option):
    graphs = []

    if not tickers:
        graphs.append(html.H3(
            "Select a stock ticker.",
            style={'marginTop': 20, 'marginBottom': 20}
        ))
    else:
        for i, ticker in enumerate(tickers):
            dff = df[df['Stock'] == ticker]

            candlestick = {
                'x': dff['Date'],
                'open': dff['Open'],
                'high': dff['High'],
                'low': dff['Low'],
                'close': dff['Close'],
                'type': 'candlestick',
                'name': ticker,
                'legendgroup': ticker
            }

            bollinger_traces = [{
                'x': dff['Date'], 'y': y,
                'type': 'scatter', 'mode': 'lines',
                'line': {'width': 1, 'color': colorscale[(i * 2) % len(colorscale)]},
                'hoverinfo': 'none',
                'legendgroup': ticker,
                'showlegend': True if i == 0 else False,
                'name': '{} - bollinger bands'.format(ticker)
            } for i, y in enumerate(bbands(dff.Close))]

            layout = {
                'margin': {'b': 20, 'r': 10, 'l': 60, 't': 20},
                'legend': {'x': 0}
            }

            if 'rangeslider' in rangeslider_option:
                layout['xaxis'] = {'rangeslider': {'visible': True}}
            else:
                layout['xaxis'] = {'rangeslider': {'visible': False}}

            graphs.append(dcc.Graph(
                id=ticker,
                figure={'data': [candlestick] + bollinger_traces, 'layout': layout}
            ))

    return graphs

if __name__ == '__main__':
    # app.run_server(debug=True)
    app.run_server(debug=True, host='0.0.0.0', port=9000)

La aplicación permite ver a través de un grafico de candlestick y bandas de Bollinger como evolucionaron en un periodo de tiempo, los precios para distintos stock. Con la app se da la opción de añadir o quitar el rangeslider, el cual permite escoger un rango de tiempo especifio en el grafico. La desventaja del rangeslider, es que solo permite ver un rango fijo de fechas, por lo que al desactivar la opción, se puede hacer zoom en areas especificas del graficos para ver con mayor detalle datos de interes. 

La aplicación al ser ejecutada, luce de la siguiente manera:


```{figure} C:/Users/kaes1/Documents/DataViz/finn1.png
---
height: 400px
name: directive-fig
---
App Financiera - Sin Rangeslider
```

```{figure} C:/Users/kaes1/Documents/DataViz/finn2.png
---
height: 400px
name: directive-fig
---
App Financiera - Con Rangeslider
```

El paso a paso para realizar la dockerización de la app es el siguiente:



1. Crear un entorno virtual de Python:

In [ ]:
conda create --name finn_venv python=3.9

2. Ejecutar la aplicación Dash dentro del entorno virtual:

In [ ]:
python index.py

Para este paso se descomenta la linea de codigo "# app.run_server(debug=True)", y se comenta " app.run_server(debug=True, host='0.0.0.0', port=9000)". En este paso se verifican las librerias que se deben instalar en el ambiente para que la app ejecute correctamente. Tambien se debe crear una carpeta para ubicar solo el archivo .py, y los demás archivos necesarios para crear la imagen de la app.

3. Crear un archivo de requerimientos:

In [ ]:
pip list --format=freeze > requirements.txt

4. Crear un archivo Dockerfile en la carpeta donde se encuentra el archivo .py:

In [ ]:
FROM python:3.9.17

COPY ./requirements.txt /requirements.txt

RUN pip install --upgrade pip
RUN pip install --no-cache-dir --upgrade -r /requirements.txt

COPY ./index.py /index.py

CMD ["python", "index.py"]

5. Crear un archivo README:

In [ ]:
# Execute the following commands to Dockerize the Dash application

docker build -t dash_finn .
docker run -h localhost -p 9002:9000 -d --name dash_finn dash_finn

6. Crear la imagen Docker:

In [ ]:
docker build -t dash_finn .

7. Ejecutar la imagen Docker:

In [ ]:
docker run -h localhost -p 9002:9000 -d --name dash_finn dash_finn